In [3]:
import json
import pandas as pd
import numpy as np
import datetime 
from bs4 import BeautifulSoup

#pd.set_option("display.max_rows", None)

In [4]:
def niceText(html):
    soup = BeautifulSoup(html, features="html.parser")

    # kill all script and style elements
    for script in soup(["script", "style"]):
        script.extract()    # rip it out

    # get text
    text = soup.get_text()

    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    text = '\n'.join(chunk for chunk in chunks if chunk)

    return(text)

In [5]:
def miniParse(file):
    f = open (file, "r")
    data = json.loads(f.read())
    f.close()

    return pd.DataFrame([i['_source'] for i in data['hits']['hits']])

In [6]:
df_s = []
for file in ['data/noticias/response0-1000.json', 'data/noticias/response1000-2000.json', 'data/noticias/response2000-3000.json', 'data/noticias/response3000-4000.json']:
    df_s.append(miniParse(file))
df = pd.concat(df_s).drop(['tablas', 'id', 'subSeccion','fuente','autor','idSeccion','idSubSeccion','idPlantilla','epigrafe','publicada','relevancia','ultimoMinuto','fechaModificacion','temas','seccion','comentarios','tituloSEO'], axis=1).reset_index(drop=True)

df["fechaPublicacion"] = pd.to_datetime(df["fechaPublicacion"]).dt.date
df["texto"] = df["texto"].apply(niceText)

/usr/local/lib/python3.8/dist-packages/bs4/__init__.py:337: MarkupResemblesLocatorWarning: "." looks like a directory name, not markup. You may want to open a file found in this directory and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [7]:
df.to_csv('data/post_parser_emol_news.csv', index=False)